In [2]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

df = pd.read_csv('Data/Dataset.csv', decimal =".", thousands=",")
df = df.drop(['Kota','Paslon 1','Paslon 2','Total'],axis=1)

scaler = MinMaxScaler()

df_minmax = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
dfX, dfY = df_minmax.iloc[:, :-1], df_minmax.iloc[:, [-1]]
kf = RepeatedKFold(n_splits=10, n_repeats = 3 , random_state=1)
kf.get_n_splits(dfX)
y = df['Partisipasi']


# Feature Selection
Feature selection merupakan sebuah teknik untuk memilih fitur yang paling relevan untuk membangun model. Fitur yang tidak relevan akan dihapus sehingga dapat mengurangi overfitting dan meningkatkan akurasi model.  
Terdapat 3 cara untuk melakukan feature selection, yaitu:
1. Filter Method
2. Wrapper Method
3. Embedded Method

Ketiga metode tersebut akan dipakai untuk mencari subset fitur terbaik dari dataset yang digunakan.

## Subset 

Dataset demografis dapat dibagi menjadi 4 subset besar yaitu:
1. Penduduk 
2. Pendidikan
3. Ekonomi
4. Sentimen

Kombinasi dari keempat subset tersebut akan diuji sebagai acuan dalam melakukan seleksi fitur.

Untuk mempermudah proses seleksi fitur, sebuah fungsi yang menggabungkan pembuatan model dan evaluasi model akan dibuat.  
Fungsi ini akan dibuaat di Utils.py

## Eval Models

In [3]:
def eval_models(dfX,y,kf):
  evals = []
  evals.append(evaluate_linreg(dfX,y,kf))  
  evals.append(evaluate_svr(dfX,y,kf))  
  evals.append(evaluate_dt(dfX,y,kf))  
  evals.append(evaluate_rf(dfX,y,kf))  
  evals.append(evaluate_xgboost(dfX,y,kf))  
  evals.append(evaluate_catboost(dfX,y,kf))  
  evals.append(evaluate_lightgbm(dfX,y,kf))  
  dfEval = pd.DataFrame(evals,columns=["Method","RSME","MAE","R2"])
  return dfEval

def compare_models(x1,x2,y,kf):
    dfEval_1 = eval_models(x1,y,kf)
    dfEval_2 = eval_models(x2,y,kf)
    dfDiff = dfEval_1.copy()
    dfDiff["RSME Diff"] = dfEval_1["RSME"] - dfEval_2["RSME"]
    dfDiff["MAE Diff"] = dfEval_1["MAE"] - dfEval_2["MAE"]
    dfDiff["R2 Diff"] = dfEval_1["R2"] - dfEval_2["R2"]
    dfDiff.drop(["Method","RSME","MAE","R2"],axis=1,inplace=True)
    return pd.concat([dfEval_1,dfEval_2,dfDiff],axis=1)


In [4]:
Subset = ["Penduduk","Pendidikan","Ekonomi","Sentimen"]
Fitur = ['Laju Pertumbuhan', 'Laki', 'Perempuan', 'Umur', 'Umur Squared','DPT'],['APM SD', 'APM SMP', 'APM SMA', 'Literate', 'TPAK', 'pendidikan_SD','pendidikan_SMP', 'pendidikan_SMA', 'pendidikan_Universitas', 'TK', 'SD', 'SMP', 'SMA', 'SMK', 'Universitas'],['Pengeluaran Bulanan','Persentase Kemiskinan', 'Indeks Pembangunan Manusia'],['Sentiment', 'Weighted Sentiment']

In [5]:
import itertools

combinations = []

for r in range(len(Fitur)+1):
    for combination in itertools.combinations(Fitur, r):
        combinations.append(combination)

combinations_name = []

for r in range(len(Subset)+1):
    for combination in itertools.combinations(Subset, r):
        combinations_name.append(combination)

dfResults = pd.DataFrame()


In [6]:
import Utils 

for i in range(1, 5):
  dfEval = dfX[Utils.flatten(list(combinations[i]))]
  results = Utils.eval_models(dfEval,y,kf)
  name = ' '.join(list(combinations_name[i]))
  results["Subset"] = name
  dfResults = pd.concat([dfResults,results])
  print(name)
  print(results)
dfResults

Penduduk
              Method      RSME       MAE        R2    Subset
0  Linear Regression  0.039146  0.031339  0.348727  Penduduk
1                SVR  0.045557  0.036148  0.117960  Penduduk
2      Decision Tree  0.056154  0.044861 -0.340104  Penduduk
3      Random Forest  0.042081  0.033502  0.247428  Penduduk
4            XGBoost  0.047816  0.037428  0.028327  Penduduk
5           Catboost  0.043899  0.035173  0.180984  Penduduk
6           LightGBM  0.041608  0.032873  0.264244  Penduduk
Pendidikan
              Method      RSME       MAE        R2      Subset
0  Linear Regression  0.046319  0.036545  0.088201  Pendidikan
1                SVR  0.047664  0.038493  0.034482  Pendidikan
2      Decision Tree  0.054108  0.043685 -0.244225  Pendidikan
3      Random Forest  0.040805  0.032964  0.292357  Pendidikan
4            XGBoost  0.044113  0.036136  0.173002  Pendidikan
5           Catboost  0.042237  0.034181  0.241829  Pendidikan
6           LightGBM  0.044534  0.035836  0.157129 

,Method,RSME,MAE,R2,Subset
0,Linear Regression,0.039146,0.031339,0.348727,Penduduk
1,SVR,0.045557,0.036148,0.117960,Penduduk
2,Decision Tree,0.056154,0.044861,-0.340104,Penduduk
3,Random Forest,0.042081,0.033502,0.247428,Penduduk
4,XGBoost,0.047816,0.037428,0.028327,Penduduk
5,Catboost,0.043899,0.035173,0.180984,Penduduk
6,LightGBM,0.041608,0.032873,0.264244,Penduduk
0,Linear Regression,0.046319,0.036545,0.088201,Pendidikan
1,SVR,0.047664,0.038493,0.034482,Pendidikan
2,Decision Tree,0.054108,0.043685,-0.244225,Pendidikan


Program Akan dirun hingga seluruh kombinasi subset fitur terpilih telah diuji.  
Hasil yang terbaik antar model antara lain:

## Hasil Evaluasi Model dengan Subset

| Method            | RSME      | MAE       | R2         | Subset                    |
|-------------------|-----------|-----------|------------|---------------------------|
| Linear Regression | 0,0391464 | 0,0313390 | 0,3487265  | Penduduk                  |
| SVR               | 0,0455569 | 0,0361479 | 0,1179601  | Penduduk                  |
| Decision Tree     | 0,0491608 | 0,0398620 | -0,0271097 | Penduduk Ekonomi Sentimen |
| Random Forest     | 0,0394041 | 0,0316941 | 0,3401233  | Penduduk Ekonomi Sentimen |
| XGBoost           | 0,0403263 | 0,0331756 | 0,3088773  | Penduduk Ekonomi Sentimen |
| LightGBM          | 0,0380277 | 0,0298752 | 0,3854200  | Penduduk Ekonomi Sentimen |
| Catboost          | 0,0384789 | 0,0308619 | 0,3707467  | Penduduk Ekonomi Sentimen |

## Filter
Filter merupakan metode yang menggunakan teknik statistik untuk memilih fitur yang paling relevan. Filter method tidak memerlukan model untuk memilih fitur namun biasanya menghasilkan akurasi yang lebih rendah dibandingkan dengan wrapper method dan embedded method. Oleh karena itu, filter akan dijadikan acuan untuk membandingkan akurasi dari kedua metode lainnya.

## SelectKBest Mutual Information

SelectKBest merupakan salah satu metode filter yang digunakan untuk memilih fitur yang paling relevan. SelectKBest menggunakan mutual_info_regression untuk menentukan fitur yang paling relevan.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression


def select_features(X_train, y_train, X_test,k):
 fs = SelectKBest(score_func=mutual_info_regression, k=k)
 fs.fit(X_train, y_train)
 mask = fs.get_support()
 new_features = dfX.columns[mask]
 return new_features
 


X_train, X_test, y_train, y_test = train_test_split(dfX, y, test_size=0.2, random_state=1)

dfResultsKBest = pd.DataFrame()
for i in range (1,5):
  features = select_features(X_train, y_train, X_test,i)
  dfEval = dfX[features]
  results = Utils.eval_models(dfEval,y,kf)
  name = ' ,'.join(features)
  results["Attributes"] = name
  dfResultsKBest = pd.concat([dfResultsKBest,results])
  print(name)
  print(results)
dfResultsKBest


Indeks Pembangunan Manusia
              Method      RSME       MAE        R2                  Attributes
0  Linear Regression  0.046515  0.036040  0.080456  Indeks Pembangunan Manusia
1                SVR  0.048040  0.039052  0.019208  Indeks Pembangunan Manusia
2      Decision Tree  0.059062  0.046238 -0.482498  Indeks Pembangunan Manusia
3      Random Forest  0.051971  0.041027 -0.147887  Indeks Pembangunan Manusia
4            XGBoost  0.057876  0.045275 -0.423546  Indeks Pembangunan Manusia
5           Catboost  0.050399  0.039891 -0.079501  Indeks Pembangunan Manusia
6           LightGBM  0.044722  0.034718  0.149976  Indeks Pembangunan Manusia
TK ,Indeks Pembangunan Manusia
              Method      RSME       MAE        R2   
0  Linear Regression  0.045945  0.035437  0.102886  \
1                SVR  0.047248  0.037882  0.051262   
2      Decision Tree  0.060734  0.046587 -0.567649   
3      Random Forest  0.048924  0.037962 -0.017241   
4            XGBoost  0.053222  0.041591

,Method,RSME,MAE,R2,Attributes
0,Linear Regression,0.046515,0.036040,0.080456,Indeks Pembangunan Manusia
1,SVR,0.048040,0.039052,0.019208,Indeks Pembangunan Manusia
2,Decision Tree,0.059062,0.046238,-0.482498,Indeks Pembangunan Manusia
3,Random Forest,0.051971,0.041027,-0.147887,Indeks Pembangunan Manusia
4,XGBoost,0.057876,0.045275,-0.423546,Indeks Pembangunan Manusia
5,Catboost,0.050399,0.039891,-0.079501,Indeks Pembangunan Manusia
6,LightGBM,0.044722,0.034718,0.149976,Indeks Pembangunan Manusia
0,Linear Regression,0.045945,0.035437,0.102886,"TK ,Indeks Pembangunan Manusia"
1,SVR,0.047248,0.037882,0.051262,"TK ,Indeks Pembangunan Manusia"
2,Decision Tree,0.060734,0.046587,-0.567649,"TK ,Indeks Pembangunan Manusia"


Hasil Hasil terbaik SelectKBest antara lain:

### Hasil Evaluasi Model dengan SelectKBest

| Method            | RSME      | MAE       | R2         | Fitur SelectKBest                                                                                                                                                                                                                                               |
|-------------------|-----------|-----------|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Linear Regression | 0,0402219 | 0,0330630 |  0,3124502 | Laki, Perempuan, Umur, Umur Squared, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                             |
| SVR               | 0,0463914 | 0,0373972 |  0,0853518 | SD, Indeks Pembangunan Manusia                                                                                                                                                                                                                                  |
| Decision Tree     | 0,0509548 | 0,0409336 | -0,1034442 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| Random Forest     | 0,0411838 | 0,0336680 |  0,2791694 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| XGBoost           | 0,0396851 | 0,0330491 |  0,3306786 | Umur, Umur Squared, pendidikan_SD, pendidikan_SMA, SD, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT                                                                                                                              |
| LightGBM          | 0,0404047 | 0,0314712 |  0,3061873 | Laju Pertumbuhan, Laki, Perempuan, Umur, Umur Squared, APM SD, APM SMP, APM SMA, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT |
| Catboost          | 0,0393219 | 0,0324618 |  0,3428736 | Laju Pertumbuhan, Laki, Perempuan, Umur, Umur Squared, APM SD, APM SMA, Literate, pendidikan_SD, pendidikan_SMP, pendidikan_SMA, pendidikan_Universitas, TK, SD, SMP, SMK, Pengeluaran Bulanan, Persentase Kemiskinan, Indeks Pembangunan Manusia, DPT          |

Seperti Hasil Modelling, kembali terlihat bahwa model linear regression, dan model ensemble learning memiliki akurasi yang lebih baik dibandingkan dengan model lainnya.

## ReliefF 

ReliefF merupakan salah satu metode filter yang digunakan untuk memilih fitur yang paling relevan. ReliefF menggunakan teknik pengurangan dimensi berbasis nearest neighbor untuk menentukan fitur yang paling relevan.  
Implementasi ReliefF yang digunakan berasal dari library skrebate.


In [3]:
from skrebate import ReliefF

fs = ReliefF(n_features_to_select=4, n_neighbors=100, n_jobs=-1)
fs.fit(dfX.values, y)
sorted_dict = {}
for name, score in zip(dfX.columns, fs.feature_importances_):
    sorted_dict[name] = score
sorted_dict = dict(sorted(sorted_dict.items(), key=lambda item: item[1], reverse=True))
sorted_dict

{'Indeks Pembangunan Manusia': 0.13284091911102613,
 'SD': 0.12034276029063282,
 'pendidikan_SMA': 0.11226924394021717,
 'pendidikan_SD': 0.09862159114928643,
 'DPT': 0.08951284040805399,
 'Persentase Kemiskinan': 0.08670741003840189,
 'pendidikan_Universitas': 0.0770181753771101,
 'Pengeluaran Bulanan': 0.06752418168824369,
 'Literate': 0.058178733750543235,
 'Perempuan': 0.052779796704443833,
 'Laki': 0.04879084066123379,
 'TK': 0.04016274053710977,
 'TPAK': 0.03947882612619671,
 'Laju Pertumbuhan': 0.03707727177524649,
 'Umur': 0.03485968154348226,
 'Umur Squared': 0.03442564055727986,
 'SMK': 0.033480552990718025,
 'SMP': 0.02936339912835733,
 'Sentiment': 0.017851810718429697,
 'Weighted Sentiment': 0.017851810718429697,
 'APM SD': 0.015954416625980503,
 'SMA': 0.014888048240001402,
 'pendidikan_SMP': 0.013984129438205239,
 'APM SMP': 0.013054998632209899,
 'APM SMA': 0.012096499861815853,
 'Universitas': -0.013497161469518246}

Hasil diatas merupakan feature importance yang dihasilkan oleh ReliefF. Hasil tersebut akan diuji menggunakan semua model untuk melihat akurasi yang dihasilkan.

## Hasil Evaluasi Model dengan ReliefF
| Method           | RSME     | MAE      | R2        | Fitur SelectKBest                                                                                                                                                                                                                                                                 |
|------------------|----------|----------|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| LinearRegression | 0,040827 | 0,032958 |  0,291616 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |
| SVR              | 0,046285 | 0,037046 |  0,089537 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD                                                                                                                                                                                                                        |
| DecisionTree     | 0,052342 | 0,041820 | -0,164340 | Indeks Pembangunan Manusia SD                                                                                                                                                                                                                                                     |
| RandomForest     | 0,040009 | 0,032160 |  0,319714 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |
| XGBoost          | 0,042302 | 0,035010 |  0,239495 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan                                                                                                                                                   |
| LightGBM         | 0,042743 | 0,034286 |  0,223575 | Indeks Pembangunan Manusia SD                                                                                                                                                                                                                                                     |
| Catboost         | 0,039739 | 0,032746 |  0,328858 | Indeks Pembangunan Manusia SD pendidikan_SMA pendidikan_SD DPT Persentase Kemiskinan pendidikan_Universitas Pengeluaran Bulanan Literate Perempuan Laki TK TPAK Laju Pertumbuhan Umur Umur Squared SMK SMP Sentiment Weighted Sentiment APM SD SMA pendidikan_SMP APM SMP APM SMA |

Hasil reliefF jauh lebih buruk dibandingkan Hasil yang diberikan subset. 

## Wrapper Method

Seperti namanya, metode wrapper membalut model machine learning dan mengukur kinerja model berdasarkan fitur yang dipilih. Metode wrapper memilih fitur berdasarkan kinerja model yang dibalut.  
Metode wrapper identik dengan penggunaan metode optimisasi sebagai algoritma pencari fitur. Beberapa metode optimisasi yang digunakan antara lain:
1. Genetic Algorithm
2. Particle Swarm Optimization
3. Dragonfly Algorithm
4. Grey Wolf Optimizer
5. Harris Hawks Optimization


## Genetic Algorithm 1

GA merupakan teknik seleksi fitur yang menggunakan konsep algoritma genetika. Implementasi dari GA yang digunakan berasal dari library genetic_selection.  
Estimator yang dipakai adalah Linear Regression, dan Random Forest.  
Kedua estimator tersebut dipakai karena Linear regression memiliki hasil terbaik diantara model yang tidak berbasis tree dan random forest merupakan model yang seimbang dalam hal akurasi dan kecepatan.

In [3]:
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
import math


def RMSEScore(actual, predict):
  score = math.sqrt(mean_squared_error(actual,predict))
  return score

estimator = LinearRegression()

report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 

for i in range (14,15):
  model = GeneticSelectionCV(estimator,
                                cv = kf,
                                verbose = 0,
                                scoring = RMSEScore,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  model = model.fit(dfX, y)
  genfeats = dfX.columns[model.support_]
  print('Features:', genfeats)

  cv_score = model.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
  
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore

Features: Index(['APM SD', 'pendidikan_SMA', 'Universitas',
       'Indeks Pembangunan Manusia'],
      dtype='object')


In [ ]:
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor
import math


def RMSEScore(actual, predict):
  score = math.sqrt(mean_squared_error(actual,predict))
  return score

estimator = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1)

report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 

for i in range (14,15):
  model = GeneticSelectionCV(estimator,
                                cv = kf,
                                verbose = 0,
                                scoring = RMSEScore,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  model = model.fit(dfX, y)
  genfeats = dfX.columns[model.support_]
  print('Features:', genfeats)

  cv_score = model.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
  
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore